# Final Working Code For all Data collection expect Address

In [1]:
"""CODE FOR LOGIN"""
def login():
    driver.get("https://landmark.clayclerk.com/LandmarkWeb/Home/Index")
    time.sleep(6)   
    driver.find_element_by_xpath('//*[@id="bodySection"]/div/div/div[2]/div/div[7]/a/img').click() #records img
    time.sleep(3)
    driver.find_element_by_xpath('//*[@id="idAcceptYes"]').click() #ACCEPT
    time.sleep(3)
    driver.find_element_by_xpath('//*[@id="lastNumOfDays-RecordDate"]/option[2]').click() #Last 7 Days
    driver.find_element_by_xpath('//*[@id="submit-RecordDate"]').click() #SUBMIT
    time.sleep(30)
    driver.find_element_by_xpath('//*[@id="resultsTable"]/thead/tr[1]/th[9]/label/input').click()#DocType Click
    driver.find_element_by_xpath('//*[@id="resultsTable"]/thead/tr[1]/th[9]/label/input').send_keys("mortgage")#DocType edit
    time.sleep(25)

"""CODE FOR PDF address extraction"""
def PDFadd():

    try:
        #os.path.isfile(os.path.join("C:\\Users\\Dishant\\Downloads\\", "Landmark Web Official Records Search.pdf")) 
        filename = os.path.join("C:\\Users\\Dishant\\Downloads\\", "Landmark Web Official Records Search.pdf")        
        pages = convert_from_path(filename, 500)
        print("\nNumber of Pages found in PDF = ",len(pages))
        trials=0
        start = ""
        end = ""
        found = False
        for page in pages:
            add=""
            page.save('C:\\Users\\Dishant\\Downloads\\add_img.jpg', 'JPEG')
            address_txt = str(((pytesseract.image_to_string(Image.open('C:\\Users\\Dishant\\Downloads\\add_img.jpg'))))).lower()
            #print(property_value)
            try:
            
                try:

                    start = re.search('which currently has the address of\s', address_txt).span()
                    end = re.search("together\swith\sall",address_txt).span()
                    add = address_txt[start[1]:end[0]]
                    demo = add.split(",")
                    plot_street = demo[0]
                    city = demo[1]
                    state_pin = re.search("florida\s\d{5}",demo[2]).group()
                    print("Address Found : \t",add)
                    print(plot_street+"|"+city+"|"+state_pin)
                    add = plot_street+"|"+city+"|"+state_pin
                    comp_address.append(add)
                    trials+=1
                    break

                except Exception as e:

                    start = re.search('which currently has the address of\s', address_txt).span()
                    end = re.search('\(\"property\\s\\naddress\"\)',address_txt).span()
                    add = address_txt[start[1]:end[0]]
                    demo = add.split(",")
                    plot_street = demo[0]
                    city = demo[1]
                    state_pin = re.search("florida\s\d{5}",demo[2]).group()
                    print("Exception Address Found : \t",add)
                    print(plot_street+"|"+city+"|"+state_pin)
                    add = plot_street+"|"+city+"|"+state_pin
                    comp_address.append(add)
                    print(e)          
                    trials+=1
                    break

            except Exception as e:
                print("Exception raised ",e)
                trials+=1
                
            time.sleep(2)
            if(trials == 5):
                break
            
        if(add==""):
            comp_address.append("NAF")
        file_upload_API()
        os.remove('C:\\Users\\Dishant\\Downloads\\add_img.jpg')
        os.remove(filename)
    except Exception as e:
        comp_address.append("NAF")
        print("Error at end ",e)


"""FOR GOING TO WESITE FOR PROPERTY VALUE"""
def prop_val():
    driver.get("http://www.totalviewrealestate.com/")
    time.sleep(7)   
    driver.find_element_by_xpath('//*[@id="header"]/form/p/input[1]').send_keys(Address) 
    time.sleep(2)
    driver.find_element_by_xpath('//*[@id="header"]/form/p/input[2]').send_keys(City) 
    time.sleep(2)
    driver.find_element_by_xpath('//*[@id="header"]/form/p/input[3]').send_keys(pin) 
    driver.find_element_by_xpath('//*[@id="header"]/form/input').click() #SUBMIT
    time.sleep(15)
    data = driver.find_element_by_xpath('//*[@id="leftpi"]/p[2]').text
    print(data)
    lis.append(data)
    time.sleep(5)

"""FOR CLAIM VALUE CONVERTING TO INTEGER"""
def claim_val_converter(string_value):
    C_Value = ""
    for i in string_value.split(".")[0]: 
        if(i.isdigit()):

            C_Value = C_Value + i
    return int(C_Value)

"""FOR HIGH VALUATION VALUE"""
def HVVF():
    high_value = ""
    for i in HVS: 
        if(i.isdigit()):
            high_value = high_value + i
    return high_value

"""FOR LOW VALUATION VALUE"""
def LVVF():
    low_value = ""
    for i in LVS: 
        if(i.isdigit()):
            low_value = low_value + i
    return low_value
        
        
"""CODE FOR API"""
def file_upload_API():
    file_upload_url = "https://brokerapi.cravingcode.in/api/UploadCountyDocument"
    File_name = os.path.join("C:\\Users\\Dishant\\Downloads\\", "Landmark Web Official Records Search.pdf")
    
    files = {"uploadedFile": open(File_name, 'rb')}
    jsondata = {"apiKey": "1FhWHQfB8v3sEMZIp2tZ1aCOXDKmLtNRjphO8MRXbfG6RuNWrA7yxyQd2YcU9FDrMIYDagUd095QSDfS5VsijA=="}
    response = requests.post(file_upload_url, files=files, data=jsondata)
    response = json.loads(response.text)
    #print(response)
    files["uploadedFile"].close()
    link = response['Message']
    doc_link.append(link)
    
##########################################################
    
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from pdf2image import convert_from_path
from bs4 import BeautifulSoup
from datetime import date
from PIL import Image 
import pandas as pd
import pytesseract
import pyautogui
import requests
import random
import PyPDF2 
import json
import mouse
import time 
import os
import re
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

pytesseract.pytesseract.tesseract_cmd = r"C:\\Program Files (x86)\\Tesseract-OCR\\tesseract.exe"
base_dir = os.path.abspath(os.path.dirname("__file__"))

# Different Proxys : =    192.41.71.199:3128   50.246.120.125:8080     
PROXY = "192.41.71.221:3128"

options = webdriver.ChromeOptions()

appState = {
    "recentDestinations": [
        {
            "id": "Save as PDF",
            "origin": "local",
            "account": ""
        }
    ],
    "selectedDestinationId": "Save as PDF",
    "version": 2
}
profile = {'printing.print_preview_sticky_settings.appState': json.dumps(appState)}

options.add_experimental_option('prefs',  {
    "download.default_directory": base_dir,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True
    }
)
options.headless = False 
options.add_argument('--proxy-server=%s' % PROXY)
options.add_experimental_option('prefs', profile)
options.add_argument('--kiosk-printing')
path = "C:\\Users\\Dishant\\Desktop\\chromedriver\\chromedriver"

global driver , pdfno , comp_address , lis , doc_link
lis=[]
doc_link=[]
comp_address = []
driver = webdriver.Chrome(executable_path = path , options = options)

# to save data into lists
bank_name = []
def_name = []
real_value = []
doc_type = []
case_date = [] 
case_no = []

login()   #Function Calling

content = driver.page_source
soup = BeautifulSoup(content, "html.parser")

#To check total number of cases found in mortgage  
items=0 
for a in soup.findAll('td', attrs={'class':'tablecell nowordbreak sorting_1'}):
    #print(a.text)
    items+=1
print("Total number of cases in MORTGAGE = ",items)
time.sleep(5)

# to scrape data out of each file
for i in range(items):
    javaScript = 'document.getElementsByClassName("tablecell nowordbreak sorting_1")['+str(i)+'].click();'
    driver.execute_script(javaScript)

    time.sleep(10)
    cn = driver.find_element_by_xpath('//*[@id="documentInformationParent"]/table/tbody/tr[1]/td[2]').text
    case_no.append(cn)
    
    cd = driver.find_element_by_xpath('//*[@id="documentInformationParent"]/table/tbody/tr[5]/td[2]').text
    case_date.append(cd.split(" ")[0])
    
    d_t = driver.find_element_by_xpath('//*[@id="documentInformationParent"]/table/tbody/tr[10]/td[2]').text
    doc_type.append(d_t)
    
    c_v = driver.find_element_by_xpath('//*[@id="documentInformationParent"]/table/tbody/tr[14]/td[2]').text
    real_value.append(c_v)
    
    def_n = driver.find_element_by_xpath('//*[@id="documentInformationParent"]/table/tbody/tr[16]/td[2]').text
    def_name.append(def_n.replace("\n"," and "))
    
    b_n = driver.find_element_by_xpath('//*[@id="documentInformationParent"]/table/tbody/tr[18]/td[2]').text
    try:
        bank_name.append(b_n.split("\n")[1])
    except Exception as e:
        print(e)
        bank_name.append(b_n)
        
    #To download PDF and take out address
    time.sleep(15)
    driver.find_element_by_xpath('//*[@id="idPrintGroup"]').click()
    driver.find_element_by_xpath('//*[@id="DocumentPrintButtonAll"]/a').click()
    time.sleep(10)
    driver.switch_to.alert.accept()
    time.sleep(25)
    time.sleep(20)
    PDFadd()     
    time.sleep(4)
    driver.find_element_by_xpath('//*[@id="returnToSearchButton"]').click()

driver.close() #browser closed after address extraction

#To check weather all the entries are of same lenght
if(len(case_date)==items and len(case_no)==items and len(bank_name)==items and len(def_name)==items and 
   len(doc_type)==items and len(real_value)==items):
    print("SUCCESS") 
else:
    print("Check the Code Once...!!")
    
# To make DataFrame and save it as a CSV file    
Case_Data = pd.DataFrame({'Case_Number':case_no,'Case_Date':case_date,'Defendent_Name':def_name,'Bank_Name':bank_name,
                          'Real_Value':real_value,'Document_Type':doc_type,'Address':comp_address})
Case_Data.to_csv('Case_Data.csv', index = False , encoding='utf-8')


"""STARTING 2nd PHASE"""
list_HVS = []
list_HVV = []
list_LVS = []
list_LVV = []
    
for i in range(3):
    print(comp_address[i])
    if(comp_address[i]=="NAF"):
        lis.append("NAF")
        continue
    else:
        demo = comp_address[i].split("|")
        Address = demo[0]
        City = demo[1]
        pin = demo[2].split(" ")[1]
        print(Address,City,pin)
        prop_val()
driver.close()


""" FOR EXTRACTING HIGH LOW VALUE OF PROPRERTY """
for i in range(len(lis)):
    demo = lis[i]
    if(demo == "NAF"):
        HVS = "NA"
        HVV = "NA"
        LVS = "NA"
        LVV = "NA"
        print("\n")
        print("Low Value String = ",LVS)
        print("Low Value = ",LVV)
        print("High Value String = ",HVS)
        print("High Value = ",HVV)
        list_HVS.append(HVS)
        list_HVV.append(HVV)
        list_LVS.append(LVS)
        list_LVV.append(LVV)
    else:
        #print(demo)
        start = re.search("Value\sRange",demo).span()
        #print(start)
        end = re.search("Change\:\s",demo).span()
        #print(end)
        got = demo[start[1]:end[0]]
        got.split("to")
        HVS = got.split("to")[1].split("\n")[0]
        HVV = HVVF()
        LVS = got.split("to")[0].split(" ")[1]
        LVV = LVVF()
        print("\n")
        print("Low Value String = ",LVS)
        print("Low Value = ",LVV)
        print("High Value String = ",HVS)
        print("High Value = ",HVV)
        list_HVS.append(HVS)
        list_HVV.append(int(HVV))
        list_LVS.append(LVS)
        list_LVV.append(int(LVV))

"""FOR GETTING CLAIM VALUE FROM DATASET"""
dataset = pd.read_csv("C:\\Users\\Dishant\\Dishant_Try\\Scraping\\Case_Data.csv")
got_str_values = dataset["Claim_Value"]
new_int_real_value = []
for i in range(len(got_str_values)):
    got_int_value = claim_val_converter(got_str_values[i])
    new_int_real_value.append(got_int_value)
    #print(got_int_value)

    
"""FOR CALCULATIONS"""
Avg_App_V_L=[]
Avg_Est_V_L = []
Acq_Ratio_L = []


for i in range(3):
    real_value = new_int_real_value[i]
    high = list_HVV[i]
    #print(high)
    low = list_LVV[i]
    #print(low)
    if(high=="NA" or low == "NA"):
        Avg_App_V = "NA"   
        Avg_App_V_L.append(Avg_App_V) #Average Approximate Value
        Avg_Est_V = "NA" 
        Avg_Est_V_L.append(Avg_Est_V) #Average Estimated Value
        Acq_Ratio = "NA"
        Acq_Ratio_L.append(Acq_Ratio) #Acquisation Ratio
    else:
        Avg_App_V = (high+low)/2   
        Avg_App_V_L.append(Avg_App_V) #Average Approximate Value
        Avg_Est_V = Avg_App_V - real_value 
        Avg_Est_V_L.append(Avg_Est_V) #Average Estimated Value
        Acq_Ratio = real_value / Avg_App_V
        Acq_Ratio_L.append(Acq_Ratio) #Acquisation Ratio
        
        
"""ADDING AQUORED DATA TO DATAFRAME"""
dataset['Average_Approx_Value'] = Avg_App_V
dataset['Average_Estimate_Value'] = Avg_Est_V
dataset['Acquisation_ratio'] = Acq_Ratio_L
dataset['Document_Refrence_Link'] = doc_link
dataset.to_csv('UPDATED_CLAY_COUNTY.csv') 
print(dataset)
os.remove("Case_Data.csv")

Total number of cases in MORTGAGE =  42

Number of Pages found in PDF =  11
Exception raised  'NoneType' object has no attribute 'span'
Address Found : 	 7706 river ave, fleming island,
[street] [city]
florida 32003 ("property address");
{zip code]


7706 river ave| fleming island|florida 32003
list index out of range

Number of Pages found in PDF =  16
Exception raised  'NoneType' object has no attribute 'span'
Exception raised  'NoneType' object has no attribute 'span'
Exception raised  'NoneType' object has no attribute 'span'
Exception raised  'NoneType' object has no attribute 'span'
Exception raised  'NoneType' object has no attribute 'span'

Number of Pages found in PDF =  11
Exception raised  'NoneType' object has no attribute 'span'
Address Found : 	 4072 scenic dr, middleburg,

[street] [city]
florida 32068 ("property address"):

[zip code]


4072 scenic dr| middleburg|florida 32068

Number of Pages found in PDF =  16
Exception raised  'NoneType' object has no attribute 'span

KeyboardInterrupt: 

In [ ]:
# options = webdriver.ChromeOptions()
# options.add_experimental_option('prefs',  {
#     "download.default_directory": base_dir,
#     "download.prompt_for_download": False,
#     "download.directory_upgrade": True,
#     "plugins.always_open_pdf_externally": True
#     }
# )

##########################

# options = Options()
# appState = {
#     "recentDestinations": [
#         {
#             "id": "Save as PDF",
#             "origin": "local",
#             "account": ""
#         }
#     ],
#     "selectedDestinationId": "Save as PDF",
#     "version": 2,
#     "download.default_directory": "A:\\webscrappy\\orange"
# }

# profile = {"printing.print_preview_sticky_settings.appState": json.dumps(appState),
# "download.default_directory": "A:\\webscrappy\\orange"}
# options.add_experimental_option("prefs", profile)
# options.add_argument('--kiosk-printing') 
# options.add_argument('--proxy-server=%s' % PROXY)
# options.headless = False 

In [ ]:
""" TO GET CURRENT FOLDER PATH """
import os
base_dir = os.path.abspath(os.path.dirname("__file__"))
PROJECT_ROOT = os.path.dirname(os.path.abspath("__file__"))
BASE_DIR = os.path.dirname(PROJECT_ROOT)
print(base_dir)
print(PROJECT_ROOT)
print(BASE_DIR)
print(os.path.realpath("__file__") )

In [ ]:
s1="ABCDE"
for i in range(5):
    print(s1)
    s1=s1[1:]+s1[0]